# Integración con **PySpark**
Otra alternativa para poder integrar todo este proyecto en conjunto y dentro todo mismo usando python puede ser con [PySpark](https://spark.apache.org/docs/latest/api/python/index.html). A continuación se detalla lo necesario para poder trabajar con esta librería y se analiza esta opción.

## Uso
Para iniciar a usar pyspark, es necesario primero crear una sesión de la siguiente forma (es necesario tener **Java** instalado:

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/25 20:44:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/25 20:44:56 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Algunos *warnings* fueron arrojados, pero no parecen ser vitales.

Pyspark trabaja de una manera más eficiente con `DataFrames` (usados también en Pandas) y estos pueden ser obtenidos a través de archivos que contengan fuentes de datos. En este caso, utilizarémos un ejemplo básico de un geojson (json).

In [14]:
# Leyendo json en Python
import json

with open('../assets/pyspark_example.geojson', 'r') as file:
    example_geojson = json.load(file)

example_geojson

{'id': {'$oid': 'exid1'},
 'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'exftnm1',
    'tiempo': 'exfttm1',
    'geometry': {'type': 'LineString', 'coordinates': [[1, 2]]}}},
  {'type': 'Feature',
   'properties': {'name': 'exftnm2',
    'tiempo': 'exfttm2',
    'geometry': {'type': 'LineString', 'coordinates': [[3, 4]]}}}]}

In [17]:
# Leyendo JSON con PySpark
example_geojson = spark.read.json("../assets/pyspark_example.geojson")
example_geojson.printSchema()

root
 |-- _corrupt_record: string (nullable = true)



Algunos *warnings* fueron arrojados, pero no parecen ser vitales.

Una vez teniendo la sesión inicializada, se crea la función usando **UDF**: **U**ser-**D**efined **F**unction.

In [5]:
# Define a User-Defined Function (UDF) for Euclidean distance
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType

def euclidean_distance(point1, point2):
  # Assuming point1 and point2 are lists with the same length
  if len(point1) != len(point2):
    raise ValueError("Points must have the same number of dimensions")
  
  # Calculate the squared difference of each dimension
  squared_differences = [(x - y) ** 2 for x, y in zip(point1, point2)]
  # Sum the squared differences
  distance = sum(squared_differences)
  # Take the square root of the sum (Euclidean distance)
  return distance ** 0.5  # More efficient than sqrt()

# Register the UDF with a specific return type (DoubleType)
euclidean_distance_udf = udf(euclidean_distance, DoubleType())

Y ahora esto se puede calcular de la siguiente forma.

In [7]:
# Sample points as Python lists
point1 = [1, 2, 3]
point2 = [4, 5, 6]

# Import the UDF (assuming it's defined in the same Python file)
# from my_functions import euclidean_distance_udf  # Replace with your file path

# Call the UDF with the sample points
distance = euclidean_distance_udf(point1, point2)

# Print the calculated distance
print("Euclidean Distance:", distance)

PySparkTypeError: [NOT_COLUMN_OR_STR] Argument `col` should be a Column or str, got list.